In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from scipy.optimize import curve_fit

In [ ]:
file_name = input("Enter the file name: ") 
#file_name = '.txt' 

#initial_water_slope = input("Enter the initial water slope with a dot as decimal: ") 
Initial_water_slope = 0.0025 # Calculate first the initial slope of water. 

concentration = float(input("Enter the nanoparticle concentration values: ")) 
#concentration = 1.0 # mg / mL units 

required_R2 = 0.98 
#required_R2 = input("Enter a required adjusted R-squared value: ") 

# If the fitting is incorrect, lower the required_R2, the points_for_fitting or both. 
points_for_fitting = 200 
#points_for_fitting = input("Enter the number of points of the linear fitting: ") 

In [ ]:
def process_file(file_name):
    # Read the values from the txt file. 
    with open(file_name, 'r') as file:
        lines = file.readlines()

    # Filter out non-numeric lines and values > 100 (Samples don't reach such high temperatures). 
    values = []
    for line in lines:
        try:
            value = float(line.strip())
            if value <= 100:
                values.append(value)
        except ValueError:
            pass

    # Remove values where the mean of the adjacent values is lower by 0.5 units. 
    filtered_values = []
    for i in range(1, len(values)-1):
        if (values[i-1] + values[i+1]) / 2 >= values[i] - 0.5:
            filtered_values.append(values[i])

    # Try linear regressions starting from each row. 
    for i in range(len(filtered_values) - points_for_fitting):
        x_values = [0.4 * j for j in range(i, i + points_for_fitting)]
        y_values = filtered_values[i:i + points_for_fitting]

        # Perform linear regression. 
        model = LinearRegression()
        x_values_reshaped = np.array(x_values).reshape(-1, 1)
        model.fit(x_values_reshaped, y_values)

        # Calculate adjusted R-squared. 
        predictions = model.predict(x_values_reshaped)
        r2_adj = 1 - (1 - r2_score(y_values, predictions)) * ((len(y_values) - 1) / (len(y_values) - len(model.coef_) - 1))

        # If adjusted R-squared is greater than, or equal to a certain value, plot the graph and return. 
        if r2_adj >= required_R2:
            x_all = [0.4 * k for k in range(len(filtered_values))]
            regression_line = model.predict(np.array(x_all).reshape(-1, 1))

            # Save intercept and slope
            intercept = model.intercept_
            slope = model.coef_[0]

            plt.plot(x_all, filtered_values, marker='o', label='Temperature')
            plt.plot(x_all, regression_line, color='red', label='Regression Line')
            plt.xlabel('Time (s)')
            plt.ylabel('Temperature (°C)')
            plt.title('Temperature over Time with Regression Line')
            plt.legend()
            plt.grid(True)
            plt.show()

            return x_all, filtered_values, intercept, slope, r2_adj, regression_line 


    print("No regression with adjusted R-squared >= 0.99 found.")
    return None

In [ ]:
result = process_file(file_name) 

if result:
    x_all, filtered_values, intercept, slope, r2_adj, regression_line = result 
    print(f"Intercept: {intercept}") 
    print(f"Slope: {slope}") 
    print(f"Adjusted R-squared: {r2_adj}") 

In [ ]:
# Specific heat of water. 
C_H2O = 4.186 # J / (g * K) units. 

# Calculating the Specific Loss Power (SLP) index. 
SLP = (C_H2O * (slope - Initial_water_slope)) / (concentration / 1000) # Watt per gram units 

print("The SLP of this sample is: ", SLP, "W/g")